<a href="https://colab.research.google.com/github/fpaleixo/arbolom/blob/main/ARBoLoM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ASP Revision of Boolean Logical Models

#Notebook by Frederico Aleixo

In [1]:
#@title ARBoLoM directory setup
%cd "/content/drive/MyDrive/FCT/5o ano/2o semestre/arbolom"

/content/drive/MyDrive/FCT/5o ano/2o semestre/arbolom


In [3]:
#@title Install clingo
!pip install clingo

     |████████████████████████████████| 2.2 MB 12.0 MB/s 


In [ ]:
#Usage: !python corruption.py -f (FILENAME) -op (OPERATIONS) -(O)p (PROBABILITY)
#Variables: 
#FILENAME - Name of the file with the model to corrupt
#OPERATIONS - A string with one (or more) specific characters, denoting which corruptions to apply. These characters are 'f','e','r' and 'a'. 'fera' would be the full string, representing that (f)unction change, (e)dge flip, edge (r)emove and edge (a)dd will all be applied.
#O - A character that can take one of four possible values: 'f','e','r' and 'a' (followed by 'p'). -fp would change the probability of function change to occur, -ep of edge removal, etc
#PROBABILITY - A float from 0.0 to 1.0 denoting the probability of a given corruption to occur. For example, -ap 0.5 would change the add edge operation's probability to 50%

!python corruption.py -op fra -f "./simple_models/9.bnet" 

In [ ]:
#Usage: !python conversion.py -f (FILENAME)
#Variables: 
#FILENAME - Path of file containing Boolean model in the BCF format to convert to ASP.

!python conversion.py -f "./simple_models/9-corrupted (1).bnet"

In [ ]:
#Usage: $python gen_observations.py -f (FILENAME) -async -e (NUMBER OF EXPERIMENTS) -t (TIME STEPS) -as (NUMBER OF ANSWER SETS)
#Optional flags:
#-async: Produces observations using the asynchronous mode.
#Variables:
#FILENAME - Path of file containing Boolean model in the BCF format written in lp.
#NUMBER OF EXPERIMENTS - The number of experiments (sets of observations) to generate.
#TIME STEPS - The number of time steps to consider in each experiment.
#NUMBER OF ANSWER SETS - The number of answer sets to obtain.

!python gen_observations.py -f "./lp_models/4.lp" -async -t 10 -e 2

In [ ]:
#Usage: $python consistency_checking.py -f (FILENAME) -o (OBSERVATIONS) -stable -sync -async 
#Optional flags:
#-stable -> Performs consistency checking using stable state observations.
#-sync -> Performs consistency checking using synchronous observations.
#-async -> Performs consistency checking using asynchronous observations.
#Variables:
#FILENAME -> Path of file containing Boolean model in the BCF format written in lp.
#OBSERVATIONS -> Path of file containing observations written in lp. 

!python consistency_checking.py -f "./lp_models/1-corrupted.lp" -o "./lp_models/obsv/sstate/1-obs.lp" -stable